In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from PIL import Image
import os
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification, ViTFeatureExtractor

In [ ]:
epsilon = 0.03
alpha = 0.01
num_iterations = 10


kaggle_save_path = '/content/working/adversarial_images_real_final'
original_save_path = '/content/working/original_image_real_final'
os.makedirs(kaggle_save_path, exist_ok=True)
os.makedirs(original_save_path, exist_ok=True)

In [ ]:
def pgd_attack(model, images, labels, epsilon, alpha, num_iter):
    """
    Perform PGD attack on images.
    Args:
        model: Neural network model.
        images: Original images.
        labels: True labels for the images.
        epsilon: Maximum perturbation.
        alpha: Step size.
        num_iter: Number of iterations.
    """
    adv_images = images.clone().detach().requires_grad_(True).to(device)
    for _ in range(num_iter):
        # Forward pass and extract logits
        outputs = model(adv_images)['logits']
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)

        # Backward pass
        model.zero_grad()
        loss.backward()

        # Update adversarial images
        adv_images = adv_images + alpha * adv_images.grad.sign()
        delta = torch.clamp(adv_images - images, min=-epsilon, max=epsilon)
        adv_images = torch.clamp(images + delta, min=0, max=1).detach_().requires_grad_(True)

    return adv_images

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_dataset = CIFAR10(root='/kaggle/working', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

100%|██████████| 170M/170M [00:38<00:00, 4.47MB/s]


Extracting /kaggle/working/cifar-10-python.tar.gz to /kaggle/working


In [ ]:
# model = resnet18(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 10)
# model.eval()

model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
import time
start = time.time()

for i, (images, labels) in enumerate(test_loader):
  if i >= 1000:
      break
  if(i%10 == 0):
    passed_time = time.time()-start
    total_time = (passed_time * 1000)/(i+1e-10)
    print(f"On image {i}, estimated time left: {(total_time-passed_time)/60} minutes")
  images = images.to(device)
  labels = labels.to(device)

  for j, image in enumerate(images):
      # Convert tensor to PIL image
      image_np = image.cpu().detach().numpy().transpose(1, 2, 0)
      image_np = (image_np * 255).astype("uint8")
      image_pil = Image.fromarray(image_np)

      # Save original image
      original_save_path_image = os.path.join(original_save_path, f"original_image_{i * len(images) + j}.png")
      image_pil.save(original_save_path_image)
  adv_images = pgd_attack(model, images, labels, epsilon, alpha, num_iterations)

  # Convert to PIL image and save to Kaggle's working directory
  adv_image = adv_images[0].cpu().detach().numpy().transpose(1, 2, 0)
  adv_image = (adv_image * 255).astype("uint8")
  adv_image = Image.fromarray(adv_image)
  adv_image.save(os.path.join(kaggle_save_path, f"adversarial_image_{i}.png"))

print(f"Generated 500 adversarial images and saved to {kaggle_save_path}!")

In [ ]:
class_names = test_dataset.classes

# Path to adversarial images
adv_images_path = '/content/working/adversarial_images_real_final'

# Compare predictions for the first 10 images
for i, (images, labels) in enumerate(test_loader):
    if i >= 10:  # Limit to the first 10 images
        break
    images = images.to(device)
    labels = labels.to(device)

    # Debug: Check the shape and type of original image and label
    print(f"Original Image {i+1} shape: {images.shape}, Label: {labels}")

    # Load adversarial image
    adv_image_path = os.path.join(adv_images_path, f"adversarial_image_{i}.png")
    try:
        adv_image = Image.open(adv_image_path)
    except Exception as e:
        print(f"Error loading adversarial image {adv_image_path}: {e}")
        continue

    # Transform adversarial image
    adv_image = transform(adv_image).to(device)  # Apply the same transform as the original images
    adv_image = adv_image.unsqueeze(0)
    # Debug: Check the shape and type of adversarial image
    print(f"Adversarial Image {i+1} shape: {adv_image.shape}")

    # Predict original image label
    outputs_original = model(images)['logits']
    _, pred_original = torch.max(outputs_original, 1)

    # Debug: Check the predicted class index
    print(f"Predicted index (Original Image {i+1}): {pred_original.item()}")

    # Check if the predicted index is within the valid range of class_names
    if pred_original.item() >= len(class_names):
        print(f"Warning: Predicted index {pred_original.item()} is out of range!")
        continue

    pred_original_label = class_names[pred_original.item()]

    # Predict adversarial image label

    outputs_adv = model(adv_image)['logits']
    _, pred_adv = torch.max(outputs_adv, 1)


    print(f"Predicted index (Adversarial Image {i+1}): {pred_adv.item()}")


    if pred_adv.item() >= len(class_names):
        print(f"Warning: Predicted index {pred_adv.item()} is out of range!")
        continue

    pred_adv_label = class_names[pred_adv.item()]


    true_label = class_names[labels.item()]
    print(f"Image {i+1}:")
    print(f"  - True Label: {true_label}")
    print(f"  - Predicted (Original): {pred_original_label}")
    print(f"  - Predicted (Adversarial): {pred_adv_label}")

Original Image 1 shape: torch.Size([1, 3, 224, 224]), Label: tensor([3], device='cuda:0')
Adversarial Image 1 shape: torch.Size([1, 3, 224, 224])
Predicted index (Original Image 1): 281
Original Image 2 shape: torch.Size([1, 3, 224, 224]), Label: tensor([8], device='cuda:0')
Adversarial Image 2 shape: torch.Size([1, 3, 224, 224])
Predicted index (Original Image 2): 814
Original Image 3 shape: torch.Size([1, 3, 224, 224]), Label: tensor([8], device='cuda:0')
Adversarial Image 3 shape: torch.Size([1, 3, 224, 224])
Predicted index (Original Image 3): 814
Original Image 4 shape: torch.Size([1, 3, 224, 224]), Label: tensor([0], device='cuda:0')
Adversarial Image 4 shape: torch.Size([1, 3, 224, 224])
Predicted index (Original Image 4): 921
Original Image 5 shape: torch.Size([1, 3, 224, 224]), Label: tensor([6], device='cuda:0')
Adversarial Image 5 shape: torch.Size([1, 3, 224, 224])
Predicted index (Original Image 5): 32
Original Image 6 shape: torch.Size([1, 3, 224, 224]), Label: tensor([6]

In [ ]:
import shutil

# Path to the folder containing adversarial images
adv_images_path = '/kaggle/working/adversarial_images_real_final'

# Path to save the zip file
zip_path = '/kaggle/working/adversarial_images_real_final.zip'

# Zip the folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', adv_images_path)

# Confirm the zip file is created
print(f"Zip file created at: {zip_path}")
